# 用于大规模图像识别的超深卷积网络（2014）

VGG架构由牛津大学视觉几何组的Karen Simonyan和Andrew Zisserman于2014年开发，因此被命名为VGG。该模型在当时显著优于之前的模型，特别是在2014年的Imagenet挑战赛中，也称为ILSVRC。

论文地址：https://arxiv.org/abs/1409.1556

<div class="alert alert-info">
**摘要**：wy-nav-content-img其在大规模图像识别设置中的准确性的影响。我们的主要贡献是对使用非常小的（3x3）卷积滤波器的网络进行彻底评估，这表明通过将深度推进到 16-19 个权重层，可以在先前的配置上实现显著的改进。这些发现是我们 2014 年 ImageNet 挑战赛提交的基础，我们的团队分别在定位和分类赛道上获得了第一和第二名。我们还表明，我们的模型在其它数据集上具有良好的泛化能力，并取得了最先进的结果。我们已经将我们表现最好的两个 ConvNet 模型公开，以促进对计算机视觉中深度视觉表示使用的进一步研究。
</div>


## VGG网络架构

<div class="wy-nav-content-img">
    <img src="./assets/VGG_vgg_arch.png" alt="VGG 的架构图">
    <p>图1: VGG 的架构图</p>
</div>

从上图中可以看出，VGG 的整体结构是由 5 个 卷积块和 3 层全连接构成的。每个卷积块是由若干的卷积层、非线性激活层、归一化层、最大池化层组成。每一个卷积块都会在第一个 Conv2d 进行特征通道的加倍 (最后一个 ConvBlock 没有改变通道数)，然后最后加一个 MaxPooling 将 FeatureMap 的分辨率减半。

* 输入为 `224x224` 的图像。
* 卷积核的形wy-nav-content-img状为 `(2,2)`。
* 每个卷积层的通道数量为 `64 -> 128 -> 256 -> 512 -> 512`。
* VGG16 具有 16 个隐藏层（13个卷积层和3个全连接层）。
* VGG19 具有 19 个隐藏层（16个卷积层和3个全连接层）。


<div class="wy-nav-content-img">
    <img src="./assets/VGG_vgg_configuration.png" width=600px alt="VGG Configuration">
    <p>不同层数的 VGG 模型的架构配置</p>
</div>

## 关键比较

* VGG（16或19层）相比当时的其他 SOTA 网络相对更深。例如，2012年ILSVRC的获胜模型 AlexNet 仅有8层。
* 采用多个小（3x3）感受野的卷积滤波器并配合ReLU激活函数，而不是一个大的（7x7或11x11）滤波器，能够更好地学习复杂特征。较小的滤波器还意味着每层的参数更少，同时在层间引入了额外的非线性。
* 多尺度训练和推理。每张图像在多个不同尺度下训练多轮，以确保不同大小的图像具有相似的特征。
* VGG网络的一致性和简洁性使其更易于扩展或进行未来改进。

## VGG 的模型实现

In [ ]:
import torch
import torch.nn as nn
from typing import List


class VGG(nn.Module):
    def __init__(
        self,
        config,
        in_chans=3,
        num_classes=1000,
        act_layer=nn.ReLU,
        norm_layer=None,
        pool_layer=nn.MaxPool2d,
    ):
        super().__init__()

        prev_chs = in_chans
        layers: List[nn.Module] = []
        for v in config:
            if v == "M":
                # "M" 代表当前层是 MaxPooling 层
                layers += [pool_layer(kernel_size=2, stride=2)]
            else:
                # conv -> bn -> relu or conv -> relu
                conv2d = nn.Conv2d(prev_chs, v, kernel_size=3, padding=1)
                if norm_layer is not None:
                    layers += [conv2d, norm_layer(v), act_layer(inplace=True)]
                else:
                    layers += [conv2d, act_layer(inplace=True)]
                prev_chs = v

        # 特征提取层：卷积和池化层
        self.feature_extractor = nn.Sequential(*layers)

        # 池化层，自适应的把 featrue extractor 出来的 featuremap 缩放到 7x7 的大小
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))

        # 用于分类的全连接层
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # 512通道，最大池化后的空间维度为7x7
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout层，概率为0.5
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes),  # 输出层，具有'num_classes'个输出单元
        )

    def forward(self, x):
        x = self.feature_extractor(x)  # 通过特征提取层
        x = self.avgpool(x)  # 通过池化层
        x = x.view(x.size(0), -1)  # 将输出展平以传递到全连接层
        x = self.classifier(x)  # 通过分类层
        return x

In [8]:
vgg19_configuration = [
    64,
    64,
    "M",
    128,
    128,
    "M",
    256,
    256,
    256,
    256,
    "M",
    512,
    512,
    512,
    512,
    "M",
    512,
    512,
    512,
    512,
    "M",
]
model = VGG(vgg19_configuration)

model_with_bn = VGG(vgg19_configuration, norm_layer=nn.BatchNorm2d)

x = torch.randn(1, 3, 224, 224)
out = model(x)
print(f"output shape of vgg model (w/o BatchNorm) : {out.shape}")

out1 = model_with_bn(x)
print(f"output shape of vgg model with BatchNorm : {out1.shape}")

output shape of vgg model (w/o BatchNorm) : torch.Size([1, 1000])
output shape of vgg model with BatchNorm : torch.Size([1, 1000])
